在上一个笔记本中已经初步处理好了需要的全量数据，现在通过scikit-learn中的一些库函数对于借阅记录进行特征编码等，最终生成的numpy可读文件data_process.npy包含(保存在./data/processed/data_process.npy中)

1.所有物品的原始特征集item_set

2.所有用户的原始特征集user_set

3.训练集/验证集/测试集 特征向量 x_train/x_val/x_test

4.训练集/验证集/测试集 标签 y_train/y_val/y_test